In [ ]:
import numpy as np
import pandas as pd
import os
import librosa, librosa.display   
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import IPython.display as ipd
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
file = '../input/birdclef-2021/train_short_audio/acafly/XC109605.ogg'
signal, sr = librosa.load(file, sr=40200)

librosa.display.waveplot(signal, sr = sr)
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.show()

n_fft = 2048
hop_length = 512
stft = librosa.core.stft(signal, hop_length = hop_length, n_fft = n_fft)
spectrogram = np.abs(stft)
log_spectrogram  = librosa.amplitude_to_db(spectrogram)

In [ ]:
print(stft.shape)
print(spectrogram.shape)
print(log_spectrogram.shape)

In [ ]:
ipd.Audio(file)

In [ ]:
train_short='../input/birdclef-2021/train_short_audio/'

In [ ]:
train_meta=pd.read_csv('../input/birdclef-2021/train_metadata.csv')
train_label=pd.read_csv('../input/birdclef-2021/train_soundscape_labels.csv')
test=pd.read_csv('../input/birdclef-2021/test.csv')
sample=pd.read_csv('../input/birdclef-2021/sample_submission.csv')

In [ ]:
train_meta[0:1]

In [ ]:
train_label[0:1]

In [ ]:
Name=[]
for file in os.listdir(train_short):
    Name+=[file]
    
print(Name)
print(len(Name))

In [ ]:
N=[]
for i in range(len(Name)):
    N+=[i]
    
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

def mapper(value):
    return reverse_mapping[value]

In [ ]:
signal, sr = librosa.load('../input/birdclef-2021/train_short_audio/acafly/XC109605.ogg', sr=40200)
n_fft = 2048
hop_length = 512

stft = librosa.core.stft(signal, hop_length = hop_length, n_fft = n_fft)
spectrogram = np.abs(stft)
obj1 = Image.fromarray(np.uint8(spectrogram))
obj2 = np.asarray(obj1.resize((96,32)))
print(obj2.shape)

plt.figure(figsize=(18,6), dpi=50)
plt.imshow(obj2.reshape(32,96))
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
trainset=[]
trainlabel=[]
count=0
n_fft = 2048
hop_length = 512

for file in tqdm(Name):
    path=os.path.join(train_short,file)
    m=len(os.listdir(path))
    for i in range(m//50):
        im = os.listdir(path)[i]
        signal, sr = librosa.load(os.path.join(path,im), sr=40200)
        stft = librosa.core.stft(signal, hop_length = hop_length, n_fft = n_fft)
        spectrogram = np.abs(stft)
        obj1 = Image.fromarray(np.uint8(spectrogram))
        obj2 = np.asarray(obj1.resize((96,32)))
        trainset.append(obj2)
        trainlabel.append(count)
    count=count+1

In [ ]:
data=(np.array(trainset)).reshape(-1,32,32,3)
labels0=np.array(trainlabel)
print(data.shape)
print(labels0.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
labels1=to_categorical(labels0)
data=np.array(data)
labels=np.array(labels1)

In [ ]:
trainx,testx,trainy,testy=train_test_split(data,labels,test_size=0.2,random_state=44)

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,
                        width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="nearest")

In [ ]:
pretrained_model3 = tf.keras.applications.DenseNet201(input_shape=(32,32,3),include_top=False,weights='imagenet',pooling='avg')
pretrained_model3.trainable = False

In [ ]:
inputs3 = pretrained_model3.input
x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model3.output)
outputs3 = tf.keras.layers.Dense(397, activation='softmax')(x3)
model = tf.keras.Model(inputs=inputs3, outputs=outputs3)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
his=model.fit(datagen.flow(trainx,trainy,batch_size=32),validation_data=(testx,testy),epochs=200)

In [ ]:
y_pred=model.predict(testx)
pred=np.argmax(y_pred,axis=1)
ground = np.argmax(testy,axis=1)
print(classification_report(ground,pred))

In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
pred2=model.predict(data)
print(data.shape)
print(pred2.shape)

PRED=[]
for item in pred2:
    value2=np.argmax(item)      
    PRED+=[value2]

In [ ]:
ANS=labels0

In [ ]:
accuracy=accuracy_score(ANS,PRED)
print(accuracy)